In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [ ]:
df = pd.read_csv('../data/tradelog_aakash.csv')

In [18]:
df.tail()

,Unnamed: 0.1,Unnamed: 0,trade_id,entry_datetime,trade_type,entry_price,exit_datetime,exit_price,pnl,sl_price,open_pct,close_pct,exit_type,vix_open,vix_close,expiry_date_exit,strike,entry_slippage,exit_slippage,holding period,final points,INDEX_LEVERAGE,PORTFOLIO,qty,ROI%
434,434,434,435,2024-12-13 13:30:00,P,24664.8500,2024-12-17 09:45:00,24564.8000,-100.0500,NaN,-0.2976,-0.4042,exit_rule,13.7500,14.3500,2024-12-19,24650,24659.9170,24563.7998,3,-96.1172,7,10000000,2838.0469,-2.7279
435,435,435,436,2024-12-17 10:15:00,C,24518.5000,2024-12-24 10:15:00,23804.9500,713.5500,NaN,2.9392,2.9107,exit_rule,14.4800,13.1600,2024-12-26,24500,24513.5963,23803.9498,7,709.6465,7,10000000,2854.9871,20.2603
436,436,436,437,2024-12-30 11:45:00,C,23874.5500,2024-12-30 12:00:00,23894.6500,-20.1000,NaN,0.0000,-0.0896,exit_rule,13.4100,13.7000,2025-01-02,23850,23869.7751,23893.6498,0,-23.8747,7,10000000,2931.9924,-0.7000
437,437,437,438,2025-01-02 10:15:00,P,23833.0000,2025-01-03 15:15:00,24008.1500,175.1500,NaN,0.8385,0.7345,exit_rule,14.4400,13.5200,2025-01-09,23850,23828.2334,24007.1498,1,178.9164,7,10000000,2937.1040,5.2550
438,438,438,439,2025-01-06 11:00:00,C,23831.0500,2025-01-08 15:00:00,23680.4500,150.6000,NaN,0.5726,0.6426,exit_rule,14.5700,14.5300,2025-01-09,23850,23826.2838,23679.4498,2,146.8340,7,10000000,2937.3443,4.3130


In [6]:
import numpy as np
df['final points'] = np.where(
    df['trade_type'] == 'C',
    df['entry_slippage'] - df['exit_slippage'],
    df['exit_slippage'] - df['entry_slippage']
)

In [8]:
INDEX_LEVERAGE = 7
PORTFOLIO = 1_00_00_000

In [14]:
df['INDEX_LEVERAGE'] = INDEX_LEVERAGE
df['PORTFOLIO'] = PORTFOLIO
df['qty'] = df['PORTFOLIO'] * df['INDEX_LEVERAGE'] / df['entry_price']

In [17]:
df['ROI%'] = df['final points'] * df['qty'] * 100 / df['PORTFOLIO']

In [19]:
df['entry_datetime'] = pd.to_datetime(df['entry_datetime'])
df['Trade Year'] = df['entry_datetime'].dt.year

In [25]:
df.head()

,Unnamed: 0.1,Unnamed: 0,trade_id,entry_datetime,trade_type,entry_price,exit_datetime,exit_price,pnl,sl_price,open_pct,close_pct,exit_type,vix_open,vix_close,expiry_date_exit,strike,entry_slippage,exit_slippage,holding period,final points,INDEX_LEVERAGE,PORTFOLIO,qty,ROI%,Trade Year,ROI% PF
0,0,0,1,2019-01-11 12:00:00,C,10775.3000,2019-01-15 09:30:00,10802.2000,-26.9000,NaN,-0.0204,-0.2459,exit_rule,15.5700,15.7300,2019-01-31,10800,10773.1449,10801.1998,3,-28.0549,7,10000000,6496.3388,-1.8225,2019,-1.8225
1,1,1,2,2019-01-15 09:45:00,P,10821.0000,2019-01-17 13:30:00,10876.2000,55.2000,NaN,0.6284,0.5194,exit_rule,15.7200,16.3000,2019-01-31,10800,10818.8358,10875.1998,2,56.3640,7,10000000,6468.9031,3.6461,2019,3.6461
2,2,2,3,2019-01-23 10:30:00,P,10923.3000,2019-01-23 14:30:00,10889.5000,-33.8000,NaN,0.0320,-0.3049,exit_rule,18.0600,17.8500,2019-01-31,10900,10921.1153,10888.4998,0,-32.6155,7,10000000,6408.3198,-2.0901,2019,-2.0901
3,3,3,4,2019-01-23 15:00:00,C,10864.8000,2019-01-25 09:30:00,10924.5000,-59.7000,NaN,0.0460,-0.5513,exit_rule,17.8300,16.9400,2019-01-31,10850,10862.6270,10923.4998,1,-60.8728,7,10000000,6442.8245,-3.9219,2019,-3.9219
4,4,4,5,2019-01-25 10:45:00,P,10907.3000,2019-01-25 14:15:00,10828.0000,-79.3000,NaN,-0.4107,-0.7243,exit_rule,16.9500,17.7600,2019-01-31,10900,10905.1185,10826.9998,0,-78.1187,7,10000000,6417.7202,-5.0134,2019,-5.0134


In [22]:
df['ROI% PF'] = df['ROI%']

In [26]:
def generate_stats(tb_expiry):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            
            "Max Drawdown",
            "ROI/DD Ratio",
            
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI% PF"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI% PF"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI% PF"] > 0]["ROI% PF"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI% PF"] < 0]["ROI% PF"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI% PF"].cumsum() - year_trades["ROI% PF"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        # variation = f'{signal_ma} , {trailing_ma}, {time_of_day}'
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            # variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI% PF"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% PF"] > 0]["ROI% PF"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% PF"] < 0]["ROI% PF"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI% PF"].cumsum() - combined_df_sorted["ROI% PF"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    # overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        # overall_variation,
    ]
    return pd.DataFrame(stats_df8)

In [27]:
stats = generate_stats(df)
stats

/tmp/ipykernel_58140/4222784236.py:46: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2019,78.3800,82,39.0244,6.4549,-2.5635,-18.1769,4.3121
2020,220.6788,62,46.7742,13.9945,-5.6110,-30.2975,7.2837
2021,109.0094,81,45.6790,7.4289,-3.7696,-51.2893,2.1254
2022,96.4451,70,42.8571,7.8374,-3.4670,-18.8990,5.1032
2023,123.2669,73,47.9452,6.1422,-2.4134,-21.4991,5.7336
2024,145.9876,69,46.3768,7.9927,-2.9670,-13.7716,10.6006
2025,9.5680,2,100.0000,4.7840,NaN,0.0000,inf
Overall,783.3359,439.0000,44.8747,8.1355,-3.3858,-51.2893,15.2729
